## Init TF

In [ ]:
from utils import *
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
# from tensorflow.python.framework.ops import disable_eager_execution
import time

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

#turn off eager execution
# disable_eager_execution() #no?

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

In [ ]:
tf.config.list_physical_devices('GPU')

## Using NumPy

In [ ]:
## from vedo import *
from utils import *
import numpy as np
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti

settings.embedWindow(backend='ipyvtk', verbose = True) #was this

try:
    plt1.closeWindow()
    print("closed")
except:
    pass

plt1 = Plotter(N=1, axes=1, bg = (0.1,0.1,0.1), bg2 = (0.3,0.3,0.3),  interactive=True)
# settings.useParallelProjection = True #makes view orthographic

## uncomment to use VOLPE dataset -----------------------------------------------------
# location = 'C:/Users/Derm/2021-03-10-16-43-50_Velodyne-VLP-16-Data_garminSignage.txt'
# cloud = np.loadtxt(open(location, "rb"), delimiter=",")
# cloud = cloud[~np.isnan(cloud).any(axis=1)] #remove all rows with NaN elements
## ------------------------------------------------------------------------------------

## uncomment to use KITTI dataset -----------------------------------------------------
basedir = 'C:/kitti/'
date = '2011_09_26'
drive = '0005'
frame_range = range(150, 151, 1)
dataset = pykitti.raw(basedir, date, drive)
velo1 = dataset.get_velo(0) # Each scan is a Nx4 array of [x,y,z,reflectance]
cloud = velo1[:,:3]
print(type(cloud))
## ------------------------------------------------------------------------------------

# make 2D sinusioal motion (for debug) ------------------------------------------------
# cloud = np.random.randn(10000,3)
# cloud[:,0] += -50*cloud[:,1] + np.random.randn()*5
# cloud[:,1] += 5
# cloud[:,1] = cloud[:,1] * 3 + 10*np.random.randn() + np.sin(cloud[:,1]*5)*10
# cloud[:,2] += 0.5*cloud[:,1] - 10
##-------------------------------------------------------------------------------------

# f =np.array([200,200,40]) #fidelity in x, y, z #takes ~30s on my dsektop
# lim = np.array([-50,50,-50,50,-10,10])

f =np.array([50,50,1]) #fidelity in x, y, z # < 5s
lim = np.array([-100,100,-100,100,-10,10])

E = subdivide_scan(cloud,plt1, bounds = lim, fid = f, draw_grid = False, show_pc = False) 
ViewInteractiveWidget(plt1.window)

## Using TensorFlow

In [1]:
## from vedo import *
from utils import *
import numpy as np
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti

settings.embedWindow(backend='ipyvtk', verbose = True) #was this

plt2 = Plotter(N=1, axes=1, bg = (0.1,0.1,0.1), bg2 = (0.3,0.3,0.3),  interactive=True)

basedir = 'C:/kitti/'
date = '2011_09_26'
drive = '0005'
frame_range = range(150, 151, 1)
dataset = pykitti.raw(basedir, date, drive)
velo1 = dataset.get_velo(0) # Each scan is a Nx4 array of [x,y,z,reflectance]
cloud = velo1[:,:3]
# print(type(cloud))

f = tf.constant([10,10,2]) #fidelity in x, y, z # < 5s
lim = tf.constant([-100.,100.,-100.,100.,-10.,10.])

E = subdivide_scan_tf(cloud,plt2, bounds = lim, fid = f, draw_grid = False, show_pc = True) 
ViewInteractiveWidget(plt2.window)

<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([23.665249 ,  0.9999643,  1.0103214], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([25.318369 ,  3.142531 ,  1.0738642], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([28.953072 , 16.444204 ,  1.3827578], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([-3.25275  , 10.039799 ,  1.1490233], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([-0.24406411,  7.712153  ,  1.126723  ], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([24.649332 , 18.603691 ,  1.3275675], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([-2.440729 , 16.835114 ,  1.2582564], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([38.254974 , 13.248639 ,  1.5939476], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, n

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

## Unit Test R() and Jacobian

In [ ]:
n_hat = np.array([1,0,0])
theta =  0.1 #np.pi/6 #rad

rot_mat_simp = R_simp(n_hat, theta)
print(rot_mat_simp)
angs = np.array([theta,0 ,0])
rot_mat = R(angs)
print(rot_mat)

print(R2Euler(rot_mat))

p_point = np.array([1,1,1]).T

J = jacobian(angs, p_point)

d_rot_mat_simp = dR_simp(n_hat,theta)
# print(d_rot_mat_simp)

## Unit test R2Euler_tf and R_tf
#### Works with single axis roation
#### Works with vectoried input
#### Solution becomes ambiguious with mutliple axis inputs

In [ ]:
angs = tf.random.normal((3,1)) * tf.constant([[0.], [0.], [1.] ]) #single axis angle input
# angs = tf.Variable([[0.],[ np.pi/2],[ 0.]]) #vector input, multiple axis rotation
print("Input angs: \n", angs.numpy())
# print("R(angs): \n", R_tf(angs).numpy())
test1 = R2Euler_tf(R_tf(angs))
print("R2Euler_tf(R(angs)): \n", test1.numpy())
# test2 = R2Euler(R(angs[:,:2]))
# print(test2)

### Unit test tfp find bins

In [ ]:
c = tf.convert_to_tensor(cloud, np.float32)#[:100]
# print("c: \n", c.numpy())
startx = -100.
stopx = 100.
numx = 10
edgesx = tf.linspace(startx, stopx, numx)
xbins = tfp.stats.find_bins(c[:,0], edgesx)
print(xbins)
starty = -100.
stopy = 100.
numy = 10
edgesy = tf.linspace(starty, stopy, numy)
ybins = tfp.stats.find_bins(c[:,1], edgesy)
print(ybins)

min_num_pts = 1000

count = 0
E = []

for x in range(numx):
    for y in range(numy):
        #only do calculations if there are a sufficicently high number of points in the bin
        xin = tf.where(xbins == x)
        if tf.shape(xin)[0] > min_num_pts:
            if tf.shape(tf.where(tf.gather(ybins, xin) == y))[0] > min_num_pts: #repeat for y points at x coord
#                 print("working", x, y)
                count += 1
# print(xin)
# print(ybins)
print(count)

In [ ]:
test = tf.linspace(1,5,5)
print(test)
ans = tf.where(test < 4)
print("ans: \n",ans)
print("\n", tf.gather(test, ans))


## Inline volumetric rendering using ipyvolume

In [ ]:
#simple demo
import ipyvolume
ds = ipyvolume.datasets.aquariusA2.fetch()
short = ds.data[:,:,:]
ipyvolume.quickvolshow(short, lighting=True)

In [ ]:
ans = np.linspace(1,12,8)
ans[2] = 0
print(ans)

test = ans[ans < 10]
print(test)
np.shape(test)[0]

print(np.median(test[:3]))

In [ ]:
# a = tf.reshape([10.,10.,2.], (3,1))
# print(type(a))
# t = tf.Tensor(a, dtype = "float32")

a = tf.constant([2.1,2.,3.])
b = tf.constant([1.,2.,3.])
tf.tensordot(a,b, axes = 0)

In [ ]:
eye = tf.eye(3)
# print(j)
Jx = tf.constant([[1.], [2.], [3.]])
J = tf.concat([eye, Jx], axis = 1)
print(J)

In [ ]:
# print(tf.sin(1.))
start = time.time()
numiter = 100
angs  = np.array([1.,0.1,0.1])
p_point = np.array([1.,2.,3.])
for _ in range(numiter):
    J = jacobian(angs, p_point);
print("took", time.time()-start, "seconds on CPU")
print(J)

In [ ]:
#TF is slower if we do them one by one BUT is waaay faster if we send them in all at once
start = time.time()
numiter = 100000
angs = tf.random.normal((3,numiter))
p_point = tf.random.normal((3,numiter))
J = jacobian_tf(angs, p_point);
print("took", time.time()-start, "seconds on GPU")
print(J.numpy())